<a href="https://colab.research.google.com/github/GuilhermeJobs/System-Search-Doc---Embeddings/blob/main/SystemSearchDoc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Imersão Inteligência Artificial 2ª Edição**

Um sistema de buscar por aproximação semântica em uma base de dados controlada desenvolvida no google colab em python ustilizando embeddings (representações vetoriais) .

A base utilizada será livros, porém, vale ressaltar que poderia ser qualquer outro conteudo.

Objetivo do projeto: Encontrar livros com títulos e resumos semelhantes a uma determinada consulta | Recomendar livros da base de dados semelhantes as consulta que um usuário inserir.

by: Guilherme Ferreira
ferreiraguilherme@discente.ufg.br

In [113]:
# O SDK do Python para a API Gemini
!pip install -U -q google-generativeai

In [114]:
# Importanndo bibliotecas que vamos usar
import pandas as pd
import numpy as np
import google.generativeai as genai
from IPython.display import clear_output

In [115]:
# Configurando nossa chave
from google.colab import userdata
API_KEY = userdata.get('SECRET_KEY')

GOOGLE_API_KEY= API_KEY
genai.configure(api_key=GOOGLE_API_KEY)

In [116]:
# Listando nosso modelos de embedding
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


Nossa base será uma lista de dicionarios. Cada dicionario será um livro com titulo, autor e resumo da trama.

In [58]:
# Documentos que serão buscados
LIVRO1 = {
    "titulo": "A Sombra do Corvo",
    "autor": "J.D. Blackwood",
    "resumo": "Em uma cidade assolada por uma série de assassinatos brutais, um detetive solitário precisa desvendar os segredos de uma sociedade secreta para deter o culpado."
}
LIVRO2 = {
    "titulo": "O Labirinto de Cristal",
    "autor": "Evelyn Arrowood",
    "resumo": "Uma jovem arqueóloga descobre um artefato misterioso que a transporta para um mundo mágico repleto de perigos e maravilhas. Ela precisa encontrar o caminho de volta para casa antes que seja tarde demais."
}
LIVRO3 = {
    "titulo": "As Crônicas de Aethelred",
    "autor": "G.R. Mallory",
    "resumo": "Uma épica saga medieval que acompanha a ascensão e queda de um rei lendário, em um mundo de batalhas sangrentas, intrigas políticas e magia ancestral."
}
LIVRO4 = {
    "titulo": "A Conspiração dos Sonhos",
    "autor": "L.C. Valentine",
    "resumo": "Em um futuro distópico onde os sonhos são controlados, um grupo de rebeldes luta para libertar a mente humana da opressão tecnológica."
}
LIVRO5 = {
    "titulo": "O Último Guardião",
    "autor": "S.K. Tremaine",
    "resumo": "O último de uma linhagem de guerreiros mágicos precisa proteger o mundo de uma antiga ameaça que despertou de seu sono milenar."
}
LIVRO6 = {
    "titulo": "A Melodia do Silêncio",
    "autor": "A.P. Blackwood",
    "resumo": "Em um mundo onde a música é proibida, uma jovem talentosa desafia as regras e embarca em uma jornada perigosa para redescobrir a beleza da melodia."
}
LIVRO7 = {
    "titulo": "O Enigma de Turing",
    "autor": "C.J. Hawthorne",
    "resumo": "Um brilhante matemático precisa decifrar um código secreto criado por uma inteligência artificial avançada, enquanto lida com seus próprios demônios internos."
}
LIVRO8 = {
    "titulo": "A Dança dos Espíritos",
    "autor": "R.D. Sterling",
    "resumo": "Uma antropóloga se aventura em uma ilha remota para estudar uma tribo com crenças ancestrais, mas logo se vê envolvida em um mundo de espíritos e mistérios."
}
LIVRO9 = {
    "titulo": "A Caçada Eterna",
    "autor": "M.L. Blackwood",
    "resumo": "Um grupo de caçadores de recompensas enfrenta perigos inimagináveis em busca de uma criatura lendária que ameaça a paz do reino."
}
LIVRO10 = {
    "titulo": "As Flores do Outono",
    "autor": "E.R. Blackwood",
    "resumo": "Em uma mansão isolada, uma família guarda segredos obscuros que vêm à tona quando um detetive chega para investigar um misterioso desaparecimento."
}

docs = [LIVRO1, LIVRO2, LIVRO3, LIVRO4, LIVRO5, LIVRO6, LIVRO7, LIVRO8, LIVRO9, LIVRO10]

In [117]:
# Testando nossa base
df = pd.DataFrame(docs)
df

,titulo,autor,resumo
0,A Sombra do Corvo,J.D. Blackwood,Em uma cidade assolada por uma série de assass...
1,O Labirinto de Cristal,Evelyn Arrowood,Uma jovem arqueóloga descobre um artefato mist...
2,As Crônicas de Aethelred,G.R. Mallory,Uma épica saga medieval que acompanha a ascens...
3,A Conspiração dos Sonhos,L.C. Valentine,Em um futuro distópico onde os sonhos são cont...
4,O Último Guardião,S.K. Tremaine,O último de uma linhagem de guerreiros mágicos...
5,A Melodia do Silêncio,A.P. Blackwood,"Em um mundo onde a música é proibida, uma jove..."
6,O Enigma de Turing,C.J. Hawthorne,Um brilhante matemático precisa decifrar um có...
7,A Dança dos Espíritos,R.D. Sterling,Uma antropóloga se aventura em uma ilha remota...
8,A Caçada Eterna,M.L. Blackwood,Um grupo de caçadores de recompensas enfrenta ...
9,As Flores do Outono,E.R. Blackwood,"Em uma mansão isolada, uma família guarda segr..."


In [118]:
# Definindo nosso modelo de embedding
model="models/embedding-001"

In [119]:
# Função para gerar embeddings (representações vetoriais) para o título e resumo de cada livro
def embed_fn(titulo, resumo):
  return genai.embed_content(model=model,
                                   content=resumo,
                                   title=titulo,
                                   task_type="retrieval_document"
                                   )["embedding"]

In [120]:
# Testando a função
df["Embedding"] = df.apply(lambda row: embed_fn(row['titulo'], row['resumo']), axis=1)
df

,titulo,autor,resumo,Embedding
0,A Sombra do Corvo,J.D. Blackwood,Em uma cidade assolada por uma série de assass...,"[0.047358997, -0.0014161671, -0.036198985, 0.0..."
1,O Labirinto de Cristal,Evelyn Arrowood,Uma jovem arqueóloga descobre um artefato mist...,"[0.018372454, -0.02478284, -0.032359883, 0.039..."
2,As Crônicas de Aethelred,G.R. Mallory,Uma épica saga medieval que acompanha a ascens...,"[0.038895596, 0.020230398, -0.028261364, -0.01..."
3,A Conspiração dos Sonhos,L.C. Valentine,Em um futuro distópico onde os sonhos são cont...,"[0.04459276, 0.02534744, -0.06542548, 0.024197..."
4,O Último Guardião,S.K. Tremaine,O último de uma linhagem de guerreiros mágicos...,"[0.057920594, 0.036906727, -0.012691284, -0.01..."
5,A Melodia do Silêncio,A.P. Blackwood,"Em um mundo onde a música é proibida, uma jove...","[0.04385162, -0.010468824, 0.0043102624, 0.054..."
6,O Enigma de Turing,C.J. Hawthorne,Um brilhante matemático precisa decifrar um có...,"[0.0015279205, -0.051632542, -0.036737777, 0.0..."
7,A Dança dos Espíritos,R.D. Sterling,Uma antropóloga se aventura em uma ilha remota...,"[0.043812715, 0.03243341, -0.01476077, 0.06136..."
8,A Caçada Eterna,M.L. Blackwood,Um grupo de caçadores de recompensas enfrenta ...,"[0.0292339, 0.023476304, -0.038677853, -0.0151..."
9,As Flores do Outono,E.R. Blackwood,"Em uma mansão isolada, uma família guarda segr...","[0.046710335, 0.008116621, -0.06122803, 0.0254..."


In [121]:
# Função para gerar embeddings para a consulta do usuário
def search_consulta(consulta, bs, model):
  embed_consulta =  genai.embed_content(model=model,
                                   content=consulta,
                                   task_type="RETRIEVAL_QUERY"
                                   )["embedding"]
  produtos_escalares = np.dot(np.stack(df['Embedding']), embed_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]['resumo']

In [122]:
# Função para retorna o titulo do livro dada a descrição no dicionário docs
def get_titulo(descricao):
  for livro in docs:
    if livro['resumo'] == descricao:
      return (livro['titulo'] + ", de " + livro['autor'])
  return None


In [126]:
# Sistema de busca de documentos ultizando embeddings
try:

  while True:
    consulta = input("Sistema de busca de documentos ultizando embeddings" +
                    "\n" +
                    "A base contém 10 livros cadatrados" +
                    "\n" +
                    "Dada uma consulta a aplicação irá retornar o livro com a representaçã vetorial mais proximo" +
                    "\n" +
                    "Digite 'sair' para encerrar"
                    "\n" +
                    "Pesquisar um livro sobre: ")
    if not consulta or consulta.upper() == "SAIR":
      clear_output()
      print("Obrigado por participar!\n")
      break
    else:
      descricao = search_consulta(consulta, df, model)
      tittulo = get_titulo(descricao)
      print("\n" +
            tittulo +
            "\n" +
            descricao)
      print("\n\nObrigado por participar!\n")
      break

except KeyboardInterrupt:
  print("\n >> Pesquisa interrompida. << \n \n")

Sistema de busca de documentos ultizando embeddings
A base contém 10 livros cadatrados
Dada uma consulta a aplicação irá retornar o livro com a representaçã vetorial mais proximo
Digite 'sair' para encerrar
Pesquisar um livro sobre: gatos

O Último Guardião, de S.K. Tremaine
O último de uma linhagem de guerreiros mágicos precisa proteger o mundo de uma antiga ameaça que despertou de seu sono milenar.


Obrigado por participar!

